In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '/home/LLM_para/para_glm4'
lora_path = './final_checkpoint_GLM4_T-E_SFT_myidea'

In [3]:
model = AutoPeftModelForCausalLM.from_pretrained(
        lora_path, trust_remote_code= True, device_map='auto')

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 10/10 [01:45<00:00, 10.60s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
max_token = 1100
# 保证特殊字符能被识别
accept_toekn = max_token - 150

tab_str = ""
schema_slicing = []
df = pd.read_csv("./mydataset_new/table_schema_Reference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"

    
print(len(schema_slicing))

df = pd.read_csv("./mydataset_new/table_schema_noReference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
#         print("+++++++++++")
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"
        
        
if len(tab_str) > 0:
    schema_slicing.append(tab_str)

print(len(schema_slicing))
# print(schema_slicing[6])
# print("#############################")
# print(schema_slicing[7])
# print(schema_slicing[8])

create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))
#     table_name_group.append()

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 699.66it/s]


6


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 702.07it/s]

8
['manufacturers', 'products', 'student', 'plays_games', 'sportsinfo', 'actor', 'entrepreneur', 'people', 'ref_hotel_star_ratings', 'ref_attraction_types', 'hotels', 'tourist_attractions', 'street_markets', 'shops', 'museums', 'royal_family', 'theme_parks', 'visits', 'photos', 'staff', 'tourist_attraction_features', 'wrestler']
['elimination', 'business', 'category', 'checkin', 'neighbourhood', 'review', 'tip', 'ref_detention_type', 'ref_incident_type', 'addresses', 'students', 'teachers', 'assessment_notes', 'behavior_incident', 'detention', 'student_addresses', 'students_in_detention', 'film']
['film_market_estimation', 'catalogs', 'catalog_structure', 'catalog_contents', 'catalog_contents_additional_attributes', 'routes', 'airports', 'stadium', 'game', 'injury_accident', 'physician', 'department', 'affiliated_with', 'trained_in', 'patient', 'nurse', 'appointment', 'prescribes', 'block', 'room']
['on_call', 'stay', 'undergoes', 'buildings', 'office_locations', 'region', 'party', 'me

In [9]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print(response)
     
    return response

# instruction = row['instruction']
# input_value = row['input']
def get_instruction(schema):
    instruction = f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema}"
    return instruction

def get_input_value(question):
    input_value = f"{question}"
    return input_value

In [10]:
df = pd.read_csv("./mydataset_new/validation_dataset_formatted_cropped.csv",encoding="utf-8")
outer_index = 0
import re
pattern = re.compile(r'-- Tables: ([\w, ]+);')
results = []
#  schema_slicing

for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    ref_tables = row['correct_tables']
    selected_table_list = []
    result_count = 0
    for schema in schema_slicing:
        result_count += 1
        if len(selected_table_list) > 0:
            selected_table =  ', '.join(selected_table_list)
        else:
            selected_table = "None"
        instruction = get_instruction(schema)
        input_value = get_input_value(question)
        messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}]
        response = predict(messages, model, tokenizer)
        if "#None#" not in response:
            match = re.search(pattern, response)
            if match:
                table_names_str = match.group(1)
                table_names = table_names_str.split(', ')
                for predict_table in table_names:
                    selected_table_list.append(predict_table)
            else:
                predict_table = "None"
    if len(selected_table_list) > 0:
        total_selected_table =  ', '.join(selected_table_list)
    else:
        total_selected_table = "None"
    print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
    print("正确表名：",ref_tables)
    print("预测表名：",total_selected_table)
    print("############################################")
    results.append([ref_tables,total_selected_table])
print(len(predict_table))

  1%|▍                                                                                  | 1/178 [00:05<16:58,  5.76s/it]

++++++++++++++++index:0++++++++++++++++++++++++
正确表名： products, manufacturers
预测表名： products, manufacturers
############################################


  1%|▉                                                                                  | 2/178 [00:11<16:35,  5.66s/it]

++++++++++++++++index:1++++++++++++++++++++++++
正确表名： products
预测表名： products, catalog_contents
############################################


  2%|█▍                                                                                 | 3/178 [00:17<16:42,  5.73s/it]

...

++++++++++++++++index:175++++++++++++++++++++++++
正确表名： railway
预测表名： railway
############################################


 99%|████████████████████████████████████████████████████████████████████████████████▌| 177/178 [17:00<00:05,  5.70s/it]

++++++++++++++++index:176++++++++++++++++++++++++
正确表名： book
预测表名： book
############################################


100%|█████████████████████████████████████████████████████████████████████████████████| 178/178 [17:06<00:00,  5.77s/it]

++++++++++++++++index:177++++++++++++++++++++++++
正确表名： book, publication
预测表名： book, publication
############################################
11


In [11]:
new_df = pd.DataFrame(results, columns = ['reference_tables','predicted_tables'])

In [12]:
print(new_df)

            reference_tables            predicted_tables
0    products, manufacturers     products, manufacturers
1                   products  products, catalog_contents
2              manufacturers               manufacturers
3              manufacturers               manufacturers
4              manufacturers               manufacturers
..                       ...                         ...
173                     mill                        mill
174                  railway                     railway
175                  railway                     railway
176                     book                        book
177        book, publication           book, publication

[178 rows x 2 columns]


In [13]:
total_samples = len(new_df)
total_accuracy = 0
filtered_accuracy = 0
total_precision = 0
total_recall = 0
count = 0
for index, row in new_df.iterrows():
#     count+=1
#     print("index:",count)
    if not row['predicted_tables'] or pd.isna(row['predicted_tables']):
        continue
    predicted_tables = row['predicted_tables'].split(",")
    reference_tables = row['reference_tables'].split(",")
    
    # Convert to lowercase and strip whitespace for comparison
    predicted_tables = [x.lower().replace("--","").replace("**","").strip() for x in predicted_tables]
    reference_tables = [x.lower().strip() for x in reference_tables]
    
    # Calculate accuracy
    if set(predicted_tables) == set(reference_tables):
        total_accuracy += 1

    # Calculate precision and recall
    true_positives = len(set(predicted_tables) & set(reference_tables))
    false_positives = len(set(predicted_tables) - set(reference_tables))
    false_negatives = len(set(reference_tables) - set(predicted_tables))

    if true_positives == len(reference_tables):
        filtered_accuracy += 1
    
    if len(predicted_tables) > 0:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
    
    total_precision += precision
    total_recall += recall

# Calculate average precision and recall
avg_precision = total_precision / total_samples
avg_recall = total_recall / total_samples

# Calculate total accuracy
accuracy = total_accuracy / total_samples
filtered_accuracy = filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.9157303370786517
Filtered Accuracy: 0.9662921348314607
Average Precision: 0.9578651685393258
Average Recall: 0.9747191011235955


In [15]:
new_df.to_csv("./predicted.csv",encoding="utf-8")